In [1]:
#Authored by Ben Harris james.ben.harris@gmail.com
#6/27/2018

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
from scipy.stats import skew,norm
from pandas.tools.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso,LassoCV,RidgeCV,RidgeClassifierCV,RidgeClassifier)
from sklearn.feature_selection import RFE, f_regression
from scipy.stats import boxcox
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv(r"C:\Users\james\OneDrive\Documents\Projects\Court Citations\Data\Muni_Court_Citations_2.csv", encoding = "ISO-8859-1")

C:\Users\james\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,20,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df[df['Docketuid'].duplicated(keep=False)]

,Agencyid,Amountpaid,Appeardate,Accident,Caseno,Casestatus,Chargefileddate,Council District,Dateentered,Docketuid,...,Violation Address Rev,Violation Description,Violation Subtype,Violation Type,Violation Year,Violation YrMo,Warrantissueddate,Warranttype,X Coord,Y Coord


In [5]:
#No Duplicate Docket ID were found.

In [6]:
#Looking for Duplicate Court Numbers
df[df['Caseno'].duplicated(keep=False)]

,Agencyid,Amountpaid,Appeardate,Accident,Caseno,Casestatus,Chargefileddate,Council District,Dateentered,Docketuid,...,Violation Address Rev,Violation Description,Violation Subtype,Violation Type,Violation Year,Violation YrMo,Warrantissueddate,Warranttype,X Coord,Y Coord
147763,1.0,0,NaN,Y,0000mc14058,NaN,10/17/2014 0:00,NaN,10/17/2014 0:00,44345479,...,3000 S SHERIDAY RD,NO PROOF - LIABILITY INSURANCE,INSURANCE,NON-MOVING,2014,2014-10,NaN,NaN,NaN,NaN
194416,1.0,0,NaN,N,0000mc16001,NaN,2/11/2016 0:00,1.0,2/11/2016 0:00,45255346,...,1000 E PINE ST,ATTENDED ACCIDENT - HIT & RUN,TRAFFIC,NON-MOVING,2015,2015-09,NaN,NaN,-95.979956,36.176893
311923,1.0,0,NaN,Y,0000mc14058,NaN,10/17/2014 0:00,NaN,10/17/2014 0:00,44345521,...,3000 S SHERIDAY RD,NO PROOF - LIABILITY INSURANCE,INSURANCE,NON-MOVING,2014,2014-10,10/17/2014 0:00,WI,NaN,NaN
347123,1.0,0,NaN,N,0000mc16001,NaN,2/11/2016 0:00,NaN,2/11/2016 0:00,45255382,...,NaN,ATTENDED ACCIDENT - HIT & RUN,TRAFFIC,NON-MOVING,2015,2015-09,2/11/2016 0:00,WI,NaN,NaN


In [7]:
#Found two pairs of duplicate court numbers. 
#The second record of each court number is issueing a warrant.  Will leave because of additional information.

In [8]:
#Number of Columns
len(df.columns)

38

In [9]:
#Looking at Distribution of numberic columns
df.describe()

,Agencyid,Amountpaid,Council District,Docketuid,Fatality,Fineamount,Nbr of Days Ticket Held,Nbr_of_Days_To_Process,Number of Records,Violation Year,X Coord,Y Coord
count,379346.000000,387079.000000,310267.000000,3.870790e+05,0.0,387079.000000,387079.000000,387079.000000,387079.0,387079.000000,332616.000000,332616.000000
mean,1.034581,105.051501,4.955216,4.391256e+07,NaN,109.210456,3.586841,7.149427,1.0,2013.902960,-95.923272,36.124373
std,0.300094,109.938233,2.465752,1.204679e+06,NaN,101.471113,13.883952,56.194207,0.0,1.526523,0.059672,0.052881
min,1.000000,0.000000,1.000000,3.764143e+07,NaN,0.000000,-1935.000000,-352.000000,1.0,2012.000000,-96.414053,35.856653
25%,1.000000,0.000000,3.000000,4.287462e+07,NaN,0.000000,0.000000,4.000000,1.0,2013.000000,-95.960906,36.079340
50%,1.000000,100.000000,5.000000,4.389680e+07,NaN,150.000000,1.000000,6.000000,1.0,2014.000000,-95.922083,36.128804
75%,1.000000,150.000000,7.000000,4.497237e+07,NaN,150.000000,5.000000,7.000000,1.0,2015.000000,-95.886308,36.160607
max,4.000000,990.000000,9.000000,4.598465e+07,NaN,9767.000000,1833.000000,32874.000000,1.0,2017.000000,-95.491335,36.509993


In [10]:
#Looking for Categorical Columns
df.dtypes

Agencyid                   float64
Amountpaid                   int64
Appeardate                  object
Accident                    object
Caseno                      object
Casestatus                  object
Chargefileddate             object
Council District           float64
Dateentered                 object
Docketuid                    int64
Fatality                   float64
Fineamount                   int64
Fine Paid                   object
Injury                      object
Juvenile                    object
Nbr of Days Ticket Held      int64
Nbr_of_Days_To_Process       int64
Number of Records            int64
Officerdiv                  object
OfficerNo2                  object
Section                     object
SEX                         object
Seat Belt                   object
Type2                       object
Type3                       object
Violationdate               object
Vehicle Insured             object
Violation Address           object
Violation Address Re

In [11]:
Clean_step1 = df.drop(['Warranttype','Violation Description','Casestatus','Caseno','X Coord','Y Coord','Docketuid','Violationdate','Violation Type','Dateentered','Chargefileddate','Appeardate','Warrantissueddate','Violation Address Rev','Violation Address','Amountpaid','Violation Year','Fatality','Nbr of Days Ticket Held','Nbr_of_Days_To_Process'], axis = 1)

In [12]:
#Get Count of unique values in each column
Clean_step1.nunique()

Agencyid                3
Accident                2
Council District        9
Fineamount            110
Fine Paid               2
Injury                  2
Juvenile                2
Number of Records       1
Officerdiv            392
OfficerNo2           1060
Section               344
SEX                     3
Seat Belt               3
Type2                 106
Type3                  99
Vehicle Insured         3
Violation Subtype      18
Violation YrMo         64
dtype: int64

In [13]:
#Removing YYYY so that the information I'm using to train the model won't be biased in the future.
Clean_step1['Violation YrMo'] = Clean_step1['Violation YrMo'].str.split('-', 1).str[1]

In [14]:
#Removing violations outside of Tulsa
Clean_step2 = Clean_step1[np.isfinite(Clean_step1['Council District'])]
#Create Clean data for training by removing the nulls from the depedent variable Fine Paid
Clean_step3 = Clean_step2[pd.notnull(Clean_step2['Fine Paid'])]
del Clean_step2

In [15]:
#Convert Cateogrical to dummies
Clean_step4 = pd.get_dummies(Clean_step3)
del Clean_step3

In [16]:
#We have completed the cleaning phase.
#Next we run a deep neural network to see if we can predict paid versus unpaid values
#For speed I used https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/ as a reference

In [17]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Activation
seed = 7
numpy.random.seed(seed)

def dropy(df,y):
    try:
        return df.drop(y, 1)
    except:
        return df

def fixData(df):
    df = df.loc[:, df.isnull().mean() < .1]
    df = df.dropna()#.drop('Doc ID',1)
    df_fixed = pd.DataFrame(np.sqrt(df),columns=df.columns)
    df_fixed.index = df.index
    return df_fixed

C:\Users\james\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
#Remove y from X
y = Clean_step4['Fine Paid_Y']
X = dropy(Clean_step4,'Fine Paid_Y')
del Clean_step4
X = dropy(X,'Fine Paid_N')

In [19]:
#Scale and Remove NaN as well as verify X and y indexes are equal.
X = fixData(X).apply(lambda s: s[np.isfinite(s)].dropna()).dropna()
y = y[y.index.isin(X.index)]
X = X[X.index.isin(y.index)]
y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

In [20]:
#Encode y
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

In [21]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(600, input_dim=1627, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(300, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
from keras.callbacks import ModelCheckpoint
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=15, verbose=1)
filepath="CourtCitations.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
#Created Estimator with K Fold 10 for Cross Validation
#estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=15, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
  7395/199172 [>.............................] - ETA: 6:57 - loss: 0.4949 - acc: 0.7658

KeyboardInterrupt: 

In [25]:
model = estimator.fit(X,y,callbacks=callbacks_list)

Epoch 1/100
221303/221303 [==============================] - 447s 2ms/step - loss: 0.4322 - acc: 0.7982
Epoch 2/100
    75/221303 [..............................] - ETA: 8:27 - loss: 0.5176 - acc: 0.7600

C:\Users\james\Anaconda3\lib\site-packages\keras-2.1.6-py3.6.egg\keras\callbacks.py:431: RuntimeWarning: Can save best model only with val_acc available, skipping.


221303/221303 [==============================] - 442s 2ms/step - loss: 0.4209 - acc: 0.8037
Epoch 3/100
221303/221303 [==============================] - 446s 2ms/step - loss: 0.4182 - acc: 0.8058
Epoch 4/100
221303/221303 [==============================] - 445s 2ms/step - loss: 0.4165 - acc: 0.8070
Epoch 5/100
221303/221303 [==============================] - 450s 2ms/step - loss: 0.4154 - acc: 0.8079
Epoch 6/100
221303/221303 [==============================] - 448s 2ms/step - loss: 0.4146 - acc: 0.8085
Epoch 7/100
221303/221303 [==============================] - 453s 2ms/step - loss: 0.4132 - acc: 0.8090
Epoch 8/100
221303/221303 [==============================] - 450s 2ms/step - loss: 0.4133 - acc: 0.8093
Epoch 9/100
221303/221303 [==============================] - 455s 2ms/step - loss: 0.4121 - acc: 0.8100
Epoch 10/100
221303/221303 [==============================] - 453s 2ms/step - loss: 0.4115 - acc: 0.8103
Epoch 11/100
221303/221303 [==============================] - 458s 2ms/step

KeyboardInterrupt: 

In [ ]:
model.model.save('CourtCitations.h5')

In [ ]:
y_pred = estimator.predict(X)

In [ ]:
#The ability to predict whether someone will pay or not looks promising.  According to the model, it has an accuracy of around 80%.  Further Analysis is needed.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import itertools
cnf_matrix = confusion_matrix(y, y_pred)
plot_confusion_matrix(cnf_matrix,[0,1],normalize=True)   


In [ ]:
#Now we perform another analysis. Are the proportion of unpaid to paid of North Tulsa different than South Tulsa?
#Used as reference http://knowledgetack.com/python/statsmodels/proportions_ztest/

In [ ]:
NTulsa = Clean_step1[Clean_step1['Council District'] == 1].groupby(['Fine Paid'])['Number of Records'].sum()
STulsa = Clean_step1[Clean_step1['Council District'] == 7].groupby(['Fine Paid'])['Number of Records'].sum()

In [ ]:
STulsa.values.flatten()

In [ ]:
NTulsa.values.flatten()

In [ ]:
#Apply Proportion Test to get p-value and t value
import statsmodels.api as sm
z_score, p_value = sm.stats.proportions_ztest([8050, 5744], [23226, 35580])
z_score

In [ ]:
from scipy.stats import norm
norm.cdf(z_score)

In [ ]:
norm.ppf(1-(0.05/2))

In [ ]:
#Since z_score > 1.95 then the two proportions are different. Thus the unpaid proportion between north and south Tulsa are different.